In [1]:
import logging
import numpy as np
import pandas as pd
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
from plotly import graph_objs as go
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
pd.options.mode.chained_assignment = None
data = pd.read_csv("csv/82000278_Toamnei_2022_05.csv")
data

,time,latitude,longitude,altitude,timelocal,temperature,pressure,humidity,voc,noise,co2,ch2o,o3,pm1,pm25,pm10,readable time,day
0,1651363204,45.651464,25.615426,538,914160,6.73,95569,76,215841,43,599,7,20,7,9,10,01-05-22 00:00,01-05-22
1,1651363264,45.651464,25.615426,538,914220,6.71,95569,76,213691,50,601,7,20,7,9,10,01-05-22 00:01,01-05-22
2,1651363324,45.651464,25.615426,538,914280,6.70,95570,76,211822,43,601,7,20,7,9,10,01-05-22 00:02,01-05-22
3,1651363384,45.651464,25.615426,538,914340,6.69,95568,76,206437,42,600,7,20,7,9,10,01-05-22 00:03,01-05-22
4,1651363444,45.651464,25.615426,538,914400,6.67,95568,77,206428,45,602,7,20,7,9,10,01-05-22 00:04,01-05-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44511,1654041310,45.651464,25.615426,538,1585140,13.53,95191,83,208469,43,590,7,20,8,10,11,31-05-22 23:55,31-05-22
44512,1654041370,45.651464,25.615426,538,1585200,13.52,95188,83,207676,45,595,7,20,8,10,11,31-05-22 23:56,31-05-22
44513,1654041430,45.651464,25.615426,538,1585260,13.52,95190,83,206661,37,590,7,20,7,9,10,31-05-22 23:57,31-05-22
44514,1654041490,45.651464,25.615426,538,1585320,13.52,95192,83,205206,43,592,7,20,7,9,10,31-05-22 23:58,31-05-22


In [3]:
# drop Nan columns and indexes
data.dropna(axis='columns', how='all', inplace=True)
data.dropna(axis='index', how='all', inplace=True)
data

,time,latitude,longitude,altitude,timelocal,temperature,pressure,humidity,voc,noise,co2,ch2o,o3,pm1,pm25,pm10,readable time,day
0,1651363204,45.651464,25.615426,538,914160,6.73,95569,76,215841,43,599,7,20,7,9,10,01-05-22 00:00,01-05-22
1,1651363264,45.651464,25.615426,538,914220,6.71,95569,76,213691,50,601,7,20,7,9,10,01-05-22 00:01,01-05-22
2,1651363324,45.651464,25.615426,538,914280,6.70,95570,76,211822,43,601,7,20,7,9,10,01-05-22 00:02,01-05-22
3,1651363384,45.651464,25.615426,538,914340,6.69,95568,76,206437,42,600,7,20,7,9,10,01-05-22 00:03,01-05-22
4,1651363444,45.651464,25.615426,538,914400,6.67,95568,77,206428,45,602,7,20,7,9,10,01-05-22 00:04,01-05-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44511,1654041310,45.651464,25.615426,538,1585140,13.53,95191,83,208469,43,590,7,20,8,10,11,31-05-22 23:55,31-05-22
44512,1654041370,45.651464,25.615426,538,1585200,13.52,95188,83,207676,45,595,7,20,8,10,11,31-05-22 23:56,31-05-22
44513,1654041430,45.651464,25.615426,538,1585260,13.52,95190,83,206661,37,590,7,20,7,9,10,31-05-22 23:57,31-05-22
44514,1654041490,45.651464,25.615426,538,1585320,13.52,95192,83,205206,43,592,7,20,7,9,10,31-05-22 23:58,31-05-22


In [4]:
# convert to date format
data['day'] = pd.to_datetime(data['day'], dayfirst=True)
data

,time,latitude,longitude,altitude,timelocal,temperature,pressure,humidity,voc,noise,co2,ch2o,o3,pm1,pm25,pm10,readable time,day
0,1651363204,45.651464,25.615426,538,914160,6.73,95569,76,215841,43,599,7,20,7,9,10,01-05-22 00:00,2022-05-01
1,1651363264,45.651464,25.615426,538,914220,6.71,95569,76,213691,50,601,7,20,7,9,10,01-05-22 00:01,2022-05-01
2,1651363324,45.651464,25.615426,538,914280,6.70,95570,76,211822,43,601,7,20,7,9,10,01-05-22 00:02,2022-05-01
3,1651363384,45.651464,25.615426,538,914340,6.69,95568,76,206437,42,600,7,20,7,9,10,01-05-22 00:03,2022-05-01
4,1651363444,45.651464,25.615426,538,914400,6.67,95568,77,206428,45,602,7,20,7,9,10,01-05-22 00:04,2022-05-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44511,1654041310,45.651464,25.615426,538,1585140,13.53,95191,83,208469,43,590,7,20,8,10,11,31-05-22 23:55,2022-05-31
44512,1654041370,45.651464,25.615426,538,1585200,13.52,95188,83,207676,45,595,7,20,8,10,11,31-05-22 23:56,2022-05-31
44513,1654041430,45.651464,25.615426,538,1585260,13.52,95190,83,206661,37,590,7,20,7,9,10,31-05-22 23:57,2022-05-31
44514,1654041490,45.651464,25.615426,538,1585320,13.52,95192,83,205206,43,592,7,20,7,9,10,31-05-22 23:58,2022-05-31


In [5]:
# modify name with any sensor name from df
sensor_name = 'pm10'

In [6]:
# sort dates by day
data = data.sort_values(by=['day'])
print("sorted days", data.day)

sorted days 0       2022-05-01
964     2022-05-01
963     2022-05-01
962     2022-05-01
961     2022-05-01
           ...    
43551   2022-05-31
43550   2022-05-31
43549   2022-05-31
43547   2022-05-31
44515   2022-05-31
Name: day, Length: 44516, dtype: datetime64[ns]


In [7]:
group_by_df = pd.DataFrame(
    [name, group.mean()[sensor_name]] for name, group in data.groupby('day')
)

group_by_df.columns = ['day', sensor_name]
group_by_df

C:\Users\Valentin.Adam\AppData\Local\Temp\ipykernel_6708\3478699423.py:2: FutureWarning: DataFrame.mean and DataFrame.median with numeric_only=None will include datetime64 and datetime64tz columns in a future version.
  [name, group.mean()[sensor_name]] for name, group in data.groupby('day')
C:\Users\Valentin.Adam\AppData\Local\Temp\ipykernel_6708\3478699423.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  [name, group.mean()[sensor_name]] for name, group in data.groupby('day')


,day,pm10
0,2022-05-01,9.133333
1,2022-05-02,10.127689
2,2022-05-03,12.832981
3,2022-05-04,13.408419
4,2022-05-05,16.205698
5,2022-05-06,13.217573
6,2022-05-07,14.511111
7,2022-05-08,13.686806
8,2022-05-09,13.878472
9,2022-05-10,10.299514


In [8]:
# group df by day
grp_date = data.groupby('day')
# calculate mean value  for every given day
data = pd.DataFrame(grp_date.mean())
print("MEAN " + sensor_name + " values by day\n", data[sensor_name])

MEAN pm10 values by day
 day
2022-05-01     9.133333
2022-05-02    10.127689
2022-05-03    12.832981
2022-05-04    13.408419
2022-05-05    16.205698
2022-05-06    13.217573
2022-05-07    14.511111
2022-05-08    13.686806
2022-05-09    13.878472
2022-05-10    10.299514
2022-05-11    10.430556
2022-05-12    13.337266
2022-05-13    16.961057
2022-05-14    10.860125
2022-05-15    10.702778
2022-05-16    12.968728
2022-05-17    15.097917
2022-05-18     4.164583
2022-05-19     4.411397
2022-05-20     6.315753
2022-05-21     7.715972
2022-05-22     4.626129
2022-05-23     6.012500
2022-05-24    11.139777
2022-05-25    15.414465
2022-05-26     6.605556
2022-05-27     7.048611
2022-05-28     9.288194
2022-05-29     5.781250
2022-05-30     7.974306
2022-05-31    11.398611
Name: pm10, dtype: float64


In [9]:
# select needed data
data = data[[sensor_name]]
data

,pm10
day,
2022-05-01,9.133333
2022-05-02,10.127689
2022-05-03,12.832981
2022-05-04,13.408419
2022-05-05,16.205698
2022-05-06,13.217573
2022-05-07,14.511111
2022-05-08,13.686806
2022-05-09,13.878472


In [10]:
# boxplot values to eliminate outliers
upper_quartile = np.percentile(data[sensor_name], 75)
lower_quartile = np.percentile(data[sensor_name], 25)
iqr = upper_quartile - lower_quartile
upper_whisker = data[sensor_name][data[sensor_name] <= upper_quartile + 1.5 * iqr].max()
lower_whisker = data[sensor_name][data[sensor_name] >= lower_quartile - 1.5 * iqr].min()

In [11]:
print(upper_quartile)
print(lower_quartile)
print(iqr)
print(upper_whisker)
print(lower_whisker)

13.372842642715906
7.382291666666666
5.9905509760492395
16.96105702364395
4.164583333333334


In [12]:
# start using prophet
logging.getLogger().setLevel(logging.ERROR)

In [13]:
# create df for prophet
df = data.reset_index()
df.columns = ['ds', 'y']
df

,ds,y
0,2022-05-01,9.133333
1,2022-05-02,10.127689
2,2022-05-03,12.832981
3,2022-05-04,13.408419
4,2022-05-05,16.205698
5,2022-05-06,13.217573
6,2022-05-07,14.511111
7,2022-05-08,13.686806
8,2022-05-09,13.878472
9,2022-05-10,10.299514


In [14]:
X = group_by_df[['day']].values
y = group_by_df[[sensor_name]].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=False)

In [15]:
# X
# y
# X_train
# X_test
# y_train
# y_test

In [16]:
# create dataframe containing only train values
dff = pd.DataFrame(index=range(0, len(y_train)))

dff['ds'] = group_by_df['day'][:len(y_train)]
dff['y'] = group_by_df[sensor_name][:len(y_train)]
dff

,ds,y
0,2022-05-01,9.133333
1,2022-05-02,10.127689
2,2022-05-03,12.832981
3,2022-05-04,13.408419
4,2022-05-05,16.205698
5,2022-05-06,13.217573
6,2022-05-07,14.511111
7,2022-05-08,13.686806
8,2022-05-09,13.878472
9,2022-05-10,10.299514


In [17]:
m = Prophet()
# fit train values to prophet
m.fit(dff)

INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
c:\VALI\Programs\Miniconda3\envs\myenv\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
INFO:fbprophet:n_changepoints greater than number of observations. Using 19.


In [18]:
# predict whole month
future = m.make_future_dataframe(periods=len(y_test))
forecast = m.predict(future)
print('forecast', forecast)

c:\VALI\Programs\Miniconda3\envs\myenv\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)
c:\VALI\Programs\Miniconda3\envs\myenv\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  components = components.append(new_comp)


forecast            ds      trend  yhat_lower  yhat_upper  trend_lower  trend_upper  \
0  2022-05-01  13.902499    7.953405   16.033601    13.902499    13.902499   
1  2022-05-02  13.668065    9.351514   17.371330    13.668065    13.668065   
2  2022-05-03  13.433631   10.928506   18.714593    13.433631    13.433631   
3  2022-05-04  13.199197    9.347768   17.408033    13.199197    13.199197   
4  2022-05-05  12.964763    8.682552   16.577408    12.964763    12.964763   
5  2022-05-06  12.730330    9.654253   18.192287    12.730330    12.730330   
6  2022-05-07  12.495896    8.476553   16.762981    12.495896    12.495896   
7  2022-05-08  12.261462    6.339927   14.393106    12.261462    12.261462   
8  2022-05-09  12.027028    7.163285   15.561032    12.027028    12.027028   
9  2022-05-10  11.792594    9.102575   17.316550    11.792594    11.792594   
10 2022-05-11  11.558160    7.413664   15.695714    11.558160    11.558160   
11 2022-05-12  11.323727    7.110890   15.059511    11.

In [19]:
# print only values of interest
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']])

           ds       yhat  yhat_lower  yhat_upper
0  2022-05-01  11.997976    7.953405   16.033601
1  2022-05-02  13.207147    9.351514   17.371330
2  2022-05-03  14.802318   10.928506   18.714593
3  2022-05-04  13.314665    9.347768   17.408033
4  2022-05-05  12.600297    8.682552   16.577408
5  2022-05-06  13.803676    9.654253   18.192287
6  2022-05-07  12.668301    8.476553   16.762981
7  2022-05-08  10.356939    6.339927   14.393106
8  2022-05-09  11.566111    7.163285   15.561032
9  2022-05-10  13.161281    9.102575   17.316550
10 2022-05-11  11.673628    7.413664   15.695714
11 2022-05-12  10.959260    7.110890   15.059511
12 2022-05-13  12.162639    8.064273   16.240192
13 2022-05-14  11.027264    6.897523   14.861224
14 2022-05-15   8.715902    4.667400   12.799661
15 2022-05-16   9.925074    5.757807   13.896702
16 2022-05-17  11.520245    7.496530   15.565868
17 2022-05-18  10.032592    6.174449   14.182908
18 2022-05-19   9.318223    5.419357   13.360879
19 2022-05-20  10.52

In [20]:
# plot predictions
fig = plot_plotly(m, forecast)
fig.update_layout(
    title=sensor_name + ' forecast for May 2022',
    xaxis_title="Day",
    yaxis_title=sensor_name)
fig.show()

In [21]:
# check if there is seasonality+trend
fig = plot_components_plotly(m, forecast)
fig.update_layout(
    title=sensor_name + " seasonality"
)
fig.show()

c:\VALI\Programs\Miniconda3\envs\myenv\lib\site-packages\fbprophet\forecaster.py:891: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



In [22]:
# define a function to make a df containing the prediction and the actual values
def make_comparison_dataframe(historical, forecast):
    return forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(historical.set_index('ds'))

In [23]:
# modify dff so that mse can be calculated for each value of the dataframe
dff['ds'] = group_by_df['day']
dff['y'] = group_by_df[sensor_name]
dff

,ds,y
0,2022-05-01,9.133333
1,2022-05-02,10.127689
2,2022-05-03,12.832981
3,2022-05-04,13.408419
4,2022-05-05,16.205698
5,2022-05-06,13.217573
6,2022-05-07,14.511111
7,2022-05-08,13.686806
8,2022-05-09,13.878472
9,2022-05-10,10.299514


In [24]:
cmp_df = make_comparison_dataframe(df, forecast)
cmp_df

,yhat,yhat_lower,yhat_upper,y
ds,,,,
2022-05-01,11.997976,7.953405,16.033601,9.133333
2022-05-02,13.207147,9.351514,17.371330,10.127689
2022-05-03,14.802318,10.928506,18.714593,12.832981
2022-05-04,13.314665,9.347768,17.408033,13.408419
2022-05-05,12.600297,8.682552,16.577408,16.205698
2022-05-06,13.803676,9.654253,18.192287,13.217573
2022-05-07,12.668301,8.476553,16.762981,14.511111
2022-05-08,10.356939,6.339927,14.393106,13.686806
2022-05-09,11.566111,7.163285,15.561032,13.878472


In [25]:
# add new column with default value
cmp_df['outlier_detected'] = 0
for i in range(len(cmp_df)):
    # detect outliers
    if (cmp_df['y'][i] > cmp_df['yhat_upper'][i] or cmp_df['y'][i] < cmp_df['yhat_lower'][i]):
        cmp_df['outlier_detected'][i] = 1
    else:
        cmp_df['outlier_detected'][i] = 0

cmp_df

,yhat,yhat_lower,yhat_upper,y,outlier_detected
ds,,,,,
2022-05-01,11.997976,7.953405,16.033601,9.133333,0
2022-05-02,13.207147,9.351514,17.371330,10.127689,0
2022-05-03,14.802318,10.928506,18.714593,12.832981,0
2022-05-04,13.314665,9.347768,17.408033,13.408419,0
2022-05-05,12.600297,8.682552,16.577408,16.205698,0
2022-05-06,13.803676,9.654253,18.192287,13.217573,0
2022-05-07,12.668301,8.476553,16.762981,14.511111,0
2022-05-08,10.356939,6.339927,14.393106,13.686806,0
2022-05-09,11.566111,7.163285,15.561032,13.878472,0


In [38]:
# actual value
fig_data = go.Figure()
fig_data.add_trace(go.Scatter(
    x=group_by_df['day'],
    y=cmp_df['y'],
    name='y(actual value)',
    mode='lines+markers',
    line=dict(
        color='rgb(75,0,130)'),
    marker=dict(color=np.where(cmp_df['outlier_detected'] == 1, 'rgb(75,0,130)', 'rgb(75,0,130)'))))

fig_data.update_layout(title='pm10 values for May 2022', yaxis_title=sensor_name, xaxis_title='Day',
                  showlegend=True)
fig_data.show()

In [27]:
# plot forecast with upper and lower bound
fig = go.Figure()

In [28]:
# predicted value
fig.add_trace(go.Scatter(
    x=group_by_df['day'],
    y=cmp_df['yhat'],
    name='yhat(predicted value)',
    mode='lines+markers',
    line=dict(
        color='rgb(95,158,160)'),
    marker=dict(
        color='rgb(95,158,160)')
))

fig.update_layout(title='pm10 values for May 2022', yaxis_title=sensor_name, xaxis_title='Day',
                  showlegend=True)
fig.show()

In [29]:
# actual value
fig.add_trace(go.Scatter(
    x=group_by_df['day'],
    y=cmp_df['y'],
    name='y(actual value)',
    mode='lines+markers',
    line=dict(
        color='rgb(75,0,130)'),
    marker=dict(color=np.where(cmp_df['outlier_detected'] == 1, 'red', 'rgb(75,0,130)'))))

fig.update_layout(title='pm10 values for May 2022 and prediction values', yaxis_title=sensor_name, xaxis_title='Day',
                  showlegend=True)
fig.show()

In [30]:
# lower bound of predicted value
fig.add_trace(go.Scatter(
    x=group_by_df['day'],
    y=cmp_df['yhat_lower'],
    name='yhat_lower',
    mode='lines+markers',
    line=dict(
        color='rgb(205,92,92)'),
    marker=dict(
        color='rgb(205,92,92)')

))

In [31]:
# upper bound of predicted value
fig.add_trace(go.Scatter(
    x=group_by_df['day'],
    y=cmp_df['yhat_upper'],
    name='yhat_upper',
    mode='lines+markers',
    line=dict(
        color='rgb(65,105,225)'),
    marker=dict(
        color='rgb(65,105,225)')
))

fig.update_layout(title='Comparison between predicted values and real ones, including upper and lower values', yaxis_title=sensor_name, xaxis_title='Day',
                  showlegend=True)
fig.show()

In [32]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [33]:
cmp_df = cmp_df.dropna()

forecast_errors = [abs(cmp_df['y'][i] - cmp_df['yhat'][i]) for i in range(len(cmp_df))]
print('Forecast Errors: ', forecast_errors)
print('MAX Forecast Error: %s' % max(forecast_errors))
print('MIN Forecast Error: %s' % min(forecast_errors))

Forecast Errors:  [2.8646424809687563, 3.0794583691015447, 1.9693371075825112, 0.09375433443588754, 3.6054015070698053, 0.5861027029341273, 1.8428102212245534, 3.329866422662885, 2.3123614585706562, 2.8617677866801934, 1.2430728534311477, 2.3780056475716727, 4.798417850920579, 0.16713889549991912, 1.9868753534403982, 3.043654205203783, 3.5776720111786844, 5.8680083719388465, 4.9068266117125825, 4.205849482689423, 1.6702551602977653, 2.448736357917169, 2.271537240902947, 1.260569663009104, 7.022909673796477, 1.071631005724825, 1.8319544514216144, 1.5430039314243214, 0.3474212551509588, 1.3313051841488228, 3.1604401749117113]
MAX Forecast Error: 7.022909673796477
MIN Forecast Error: 0.09375433443588754


In [34]:
rmse = np.sqrt(mean_squared_error(cmp_df['y'], cmp_df['yhat']))
print("MSE is ", mean_squared_error(cmp_df['y'], cmp_df['yhat']))
print("rmse is ", rmse)
print("r2 score ", r2_score(cmp_df['y'], cmp_df['yhat']))  # around 1

MSE is  9.000845968384828
rmse is  3.000140991417708
r2 score  0.32014919901643213


In [35]:
def correlation_line(df, x, y):
    scatter_data = go.Scattergl(
        x=df[x],
        y=df[y],
        mode='markers',
        name=x + ' and ' + y + ' correlation'
    )

    layout = go.Layout(
        xaxis=dict(
            title=x
        ),
        yaxis=dict(
            title=y)
    )

    # calculate best fit line
    denominator = (df[x] ** 2).sum() - df[x].mean() * df[x].sum()
    print('denominator', denominator)
    m = ((df[y] * df[x]).sum() - df[y].mean() * df[x].sum()) / denominator
    b = ((df[y].mean() * ((df[x] ** 2).sum())) - df[x].mean() * ((df[y] * df[x]).sum())) / denominator
    best_fit_line = m * df[x] + b

    best_fit_line = go.Scattergl(
        x=df[x],
        y=best_fit_line,
        name='Line of best fit',
        line=dict(
            color='red'
        )
    )

    data = [scatter_data, best_fit_line]
    figure = go.Figure(data=data, layout=layout)

    figure.show()


# yhat and y
correlation_line(cmp_df, cmp_df.columns[0], cmp_df.columns[3])

denominator 160.9614768289398
